# Writing an OT2 Protocol using OTProto

The generic BMS and OTProto tools can be used to help write automation protocols for the Opentrons. If you will be running the same protocol a lot but with variations, you may want to look at creating an OTProto template (if one doesn't already exist). However to begin with, or for protocols which will not need to be modified with each run, it can be more useful and faster to write the protocol as a script, rather than as a template.

The tutorial here will talk through the creation of an automation protocol for setting up a protocol to mix together different types of food colouring in a 96 well plate.

## Defining the Protocol

The first step is to determine what exactly our protocol will be doing.

### Protocol behaviour

This protocol will combinatorially create two-colour mixtures of food colouring from a stock of four different food colouring with a final volume of 100 uL. Each two-colour mixture will be made in triplicate.

### Labware and materials

The protocol will use four types of food colouring:
* Green
* Blue
* Red
* Purple

The food colouring will be supplied in 1 mL tubes. The tubes will be stored in a 24 well aluminium tube rack.

The food colouring mixtures will be prepared into a Greiner 96 well flat-bottom plate.

Standard Opentrons 300 uL tips will be used.

## Setting Up

The first step is to import the BMS generic tools and the OTProto module:

In [1]:
import BiomationScripter as BMS
import BiomationScripter.OTProto as OTP

We'll also import the `math` module, as we'll use it later on

In [2]:
import math

This next step is not needed for simulating the protocol on a computer, but is required for running the protocol on the Opentrons. This step is used to point the Opentrons to the location of the BMS library. To see how BMS is loaded onto the Opentrons, see [here](https://github.com/intbio-ncl/BiomationScripterLib/blob/main/docs/OTProto.md#setting-up-the-ot-2-to-work-with-biomationscripter).

In [3]:
import sys
sys.path.insert(0, "/var/lib/jupyter/notebooks/Packages/BiomationScripterLib")

Finally, the metadata for the protocol needs to be defined. This is done through the use of a Python dictionary, as shown below:

In [4]:
metadata = {
    'protocolName': 'Colour Mixing Tutorial',
    'description': 'A protocol to create combinatorial mixtures of food colouring.',
    'author': 'Bradley Brown',
    'author-email': 'b.bradley2@newcastle.ac.uk',
    'user': 'Your Name',
    'user-email': 'name.your@email.co.uk',
    'source': 'BiomationScripter Library',
    'bms-version': '1.0.0',
    'apiLevel': '2.11',
    'robotName': 'RoBoT2' # This is the name of the OT2 you plan to run the protocol on
}

## Making the `run` Function

The rest of the Opentrons protocol must be contained within a single Python function named`run`, which takes an argument with keyword `protcol`. This argument accepts the `ProtocolContext` object generated by the Opentrons. You don't need to worry too much about how this is working for now, but you can read more [here](https://docs.opentrons.com/v2/tutorial.html#the-run-function).

For now, we will define the function and have it do nothing:

In [5]:
def run(protocol):
    pass

**NOTE:** To make the walkthrough easier to follow, the code will be shown outside of this function. It will then be packaged back into the function at the end.

In [6]:
from opentrons import simulate as OT2 # This line simulates the protocol
# Get the correct api version
protocol = OT2.get_protocol_api(metadata["apiLevel"])
# Home the pipetting head
protocol.home()

C:\Users\bradl\.opentrons\robot_settings.json not found. Loading defaults
C:\Users\bradl\.opentrons\deck_calibration.json not found. Loading defaults


## General Information

The first thing we'll do is define some of the general information for the protocol. This includes a list of source materials and their aliquot volumes, the pipettes types available, and the types of tips which will be used.

In [7]:
#######################
# General Information #
#######################

# Define the types of pipettes, and their location
Pipettes = {
    "left": "p20_single_gen2",
    "right": "p300_single_gen2"
}

# For each pipette, define the pipette tips available to them
Pipette_Tips = {
    "left": "opentrons_96_tiprack_20ul",
    "right": "opentrons_96_tiprack_300ul"
}

# List of source material

Food_Colouring = [
    "Green",
    "Blue",
    "Red",
    "Purple"
]

Food_Colouring_Aliquot_Volume = 500 # uL

## Load Pipettes

Here we'll load the pipettes - this lets the Opentrons API know which pipettes are available, and which position they're in. Note that this code uses the `protocol` object. This is the object passed in by the `run` function mentioned above. So for now, running this will cause an error (as we haven't defined `protocol`). This will become functional once we construct the whole function at the end.

In [8]:
#################
# Load Pipettes #
#################

for position in Pipettes.keys():
    protocol.load_instrument(
        Pipettes[position],
        position
    )

## Definining the Labware

Here we will define the labware (i.e the plates, tubes, and racks) required by the protocol, as defined above. It is important to determine whether you are using any 'custom' labware. Custom labware simply refers to any labware which is not included in the [Opentrons Labware Library](https://labware.opentrons.com/).

In this protocol, not including the pipette tips which will be dealt with later, we are using two types of labware:
* Aluminium 24 well tube rack: used to host the source materials (i.e. the food colouring)
* Greiner 96 well plate: used to prepare the serial dilutions

The tube rack is included in the labware library (found [here](https://labware.opentrons.com/opentrons_24_aluminumblock_nest_1.5ml_snapcap?category=aluminumBlock)), and so is not custom labware.

the Greiner 96 well plate does not exist in the labware library, and so is referred to as custom, and requires a definition file. Definition files are json files which let the Opentrons know the dimensions of the labware (in this case, a 96 well plate).

For custom labware, the [Opentrons Labware Creator](https://labware.opentrons.com/create) can be used to create definition files. Once you've created the custom labware file, follow the instructions on the creator to calibrate the labware on your robot.

For this tutorial, we have already created the definition file, which can be found [here](https://github.com/intbio-ncl/BiomationScripterLib/blob/main/Resources/For%20docs/custom_labware/greiner655087_96_wellplate_340ul.json).

To simulate the protocol, we need to supply the location of any custom labware files. This is shown below:

In [9]:
##################
# Set up labware #
##################
Custom_Labware_Directory = "../../../../Resources/For docs/custom_labware"

Then we will define the labware types using their API name. The API name is an identifier used by the Opentrons to get the desired labware. For non-custom labware, such as the aluminium tube rack, this can be found in the [Opentrons Labware Library](https://labware.opentrons.com/). For custom labware, like the 96 well plate, this is the name of the definition file, without the '.json' extension. So in this case, the API names would be:
* opentrons_24_aluminumblock_nest_1.5ml_snapcap
* greiner655087_96_wellplate_340ul

In [10]:
##################
# Set up labware #
##################
Custom_Labware_Directory = "../../../../Resources/For docs/custom_labware"

Tube_Rack_Type = "opentrons_24_aluminumblock_nest_1.5ml_snapcap"
Destination_Type = "greiner655087_96_wellplate_340ul"

Now we'll make use of the [`BMS.Labware_Layout`](https://github.com/intbio-ncl/BiomationScripterLib/blob/main/docs/BiomationScripter.md#class-labware_layout) class to help keep track of what's in our source and destination labware. This can be used later to simplify some of the liquid handling steps.

Let's first create a `Labware_Layout` object for our source labware.

In [11]:
##########################
# Set up labware layouts #
##########################

# Create the source object
Source_Labware_Layout = BMS.Labware_Layout(
    Name = "Source Labware",
    Type = Tube_Rack_Type
)

Next we'll define the format of the labware (i.e. the number of rows and columns). We can use the `OTP.get_labware_format` function to get the labware format, and then store it in the `Labware_Layout` object with the `.define_format` method.

In [12]:
##########################
# Set up labware layouts #
##########################

# Create the source object
Source_Labware_Layout = BMS.Labware_Layout(
    Name = "Source Labware",
    Type = Tube_Rack_Type
)

# Define the labware's format (i.e. number of rows and columns)
source_rows, source_columns = OTP.get_labware_format(
    labware_api_name = Source_Labware_Layout.type,
)
Source_Labware_Layout.define_format(source_rows, source_columns)

Then we do the same for the destination labware. Note that this time we need to supply `OTP.get_labware_format` with the directory (folder) containing the labware definition file. This is because the destination labware is custom labware.

In [13]:
# Create the destination object
Destination_Labware_Layout = BMS.Labware_Layout(
    Name = "Destination Labware",
    Type = Destination_Type
)

# Define the labware's format (i.e. number of rows and columns)
destination_rows, destination_columns = OTP.get_labware_format(
    labware_api_name = Destination_Labware_Layout.type,
    custom_labware_dir = Custom_Labware_Directory
)
Destination_Labware_Layout.define_format(destination_rows, destination_columns)

We can also specify which wells in the labware are available for use. This can be useful in cases where some labware, like plates, are being re-used from previous protocols, and therefore some wells are contaminated.

For this example, we'll assume that all wells for all labware are available. Information on how the `.set_available_wells()` method works can be found in [the documentation](https://github.com/intbio-ncl/BiomationScripterLib/blob/main/docs/BiomationScripter.md#class-labware_layout).

In [14]:
Source_Labware_Layout.set_available_wells()
Destination_Labware_Layout.set_available_wells()

## Define the Destination State

In this section, we'll define the final state of the destination labware. First, let's write some code to get a list of all the different two-colour mixtures which exist.

In [15]:
################################
# Define the destination state #
################################

# Set up an empty list in which the colour mixtures required will be added
Colour_Mixtures = []

# Iterate through the different ratios
# Iterate through the list of source colours provided to create the list of mixtures.
for colour_1 in Food_Colouring:
    for colour_2 in Food_Colouring:
        # Ignore situations where colour_1 and colour_2 are the same
        if colour_1 == colour_2:
            continue
        else:
            # Add the two colours to the list of mixtures to prepare
            Colour_Mixtures.append([colour_1, colour_2])

# Here, we'll print to OUT all of the mixtures which will be prepared
for c in Colour_Mixtures:
    print(c)

['Green', 'Blue']
['Green', 'Red']
['Green', 'Purple']
['Blue', 'Green']
['Blue', 'Red']
['Blue', 'Purple']
['Red', 'Green']
['Red', 'Blue']
['Red', 'Purple']
['Purple', 'Green']
['Purple', 'Blue']
['Purple', 'Red']


Now we can use this list of mixtures to define the final state of the destination labware. Note that Bbcause we've defined which wells are available, we can use the `get_next_empty_well` method, which will simply return the next available well with no content.

In [16]:
# Define the final destination state

# For 3 replicates
for rep in range(0, 3):
    # For each mixture
    for colour_1, colour_2 in Colour_Mixtures:
        
        # Get the next empty well
        well = Destination_Labware_Layout.get_next_empty_well()

        # Add the first colourant
        Destination_Labware_Layout.add_content(
            Well = well,
            Reagent = colour_1,
            Volume = 50 # uL
        )
        
        # And then the second
        Destination_Labware_Layout.add_content(
            Well = well,
            Reagent = colour_2,
            Volume = 50 # uL
        )
        
        # Can also label the well
        Destination_Labware_Layout.add_well_label(
            Well = well,
            Label = "Mixture: {}-{} Rep {}".format(colour_1, colour_2, rep)
        )

At any point, the `.print` method can be used to print to `OUT` information about a labware layout and its content.

In [17]:
Destination_Labware_Layout.print()

Information for Destination Labware
Plate Type: greiner655087_96_wellplate_340ul
Well	Volume(uL)	Liquid Class	Reagent
A1	50.0		Unknown		Green
A1	50.0		Unknown		Blue
A2	50.0		Unknown		Green
A2	50.0		Unknown		Red
A3	50.0		Unknown		Green
A3	50.0		Unknown		Purple
A4	50.0		Unknown		Blue
A4	50.0		Unknown		Green
A5	50.0		Unknown		Blue
A5	50.0		Unknown		Red
A6	50.0		Unknown		Blue
A6	50.0		Unknown		Purple
A7	50.0		Unknown		Red
A7	50.0		Unknown		Green
A8	50.0		Unknown		Red
A8	50.0		Unknown		Blue
A9	50.0		Unknown		Red
A9	50.0		Unknown		Purple
A10	50.0		Unknown		Purple
A10	50.0		Unknown		Green
A11	50.0		Unknown		Purple
A11	50.0		Unknown		Blue
A12	50.0		Unknown		Purple
A12	50.0		Unknown		Red
B1	50.0		Unknown		Green
B1	50.0		Unknown		Blue
B2	50.0		Unknown		Green
B2	50.0		Unknown		Red
B3	50.0		Unknown		Green
B3	50.0		Unknown		Purple
B4	50.0		Unknown		Blue
B4	50.0		Unknown		Green
B5	50.0		Unknown		Blue
B5	50.0		Unknown		Red
B6	50.0		Unknown		Blue
B6	50.0		Unknown		Purple
B7	50.0		Unknown		Red
B7	50.0	

'A1\t50.0\t\tUnknown\t\tGreen\nA1\t50.0\t\tUnknown\t\tBlue\nA2\t50.0\t\tUnknown\t\tGreen\nA2\t50.0\t\tUnknown\t\tRed\nA3\t50.0\t\tUnknown\t\tGreen\nA3\t50.0\t\tUnknown\t\tPurple\nA4\t50.0\t\tUnknown\t\tBlue\nA4\t50.0\t\tUnknown\t\tGreen\nA5\t50.0\t\tUnknown\t\tBlue\nA5\t50.0\t\tUnknown\t\tRed\nA6\t50.0\t\tUnknown\t\tBlue\nA6\t50.0\t\tUnknown\t\tPurple\nA7\t50.0\t\tUnknown\t\tRed\nA7\t50.0\t\tUnknown\t\tGreen\nA8\t50.0\t\tUnknown\t\tRed\nA8\t50.0\t\tUnknown\t\tBlue\nA9\t50.0\t\tUnknown\t\tRed\nA9\t50.0\t\tUnknown\t\tPurple\nA10\t50.0\t\tUnknown\t\tPurple\nA10\t50.0\t\tUnknown\t\tGreen\nA11\t50.0\t\tUnknown\t\tPurple\nA11\t50.0\t\tUnknown\t\tBlue\nA12\t50.0\t\tUnknown\t\tPurple\nA12\t50.0\t\tUnknown\t\tRed\nB1\t50.0\t\tUnknown\t\tGreen\nB1\t50.0\t\tUnknown\t\tBlue\nB2\t50.0\t\tUnknown\t\tGreen\nB2\t50.0\t\tUnknown\t\tRed\nB3\t50.0\t\tUnknown\t\tGreen\nB3\t50.0\t\tUnknown\t\tPurple\nB4\t50.0\t\tUnknown\t\tBlue\nB4\t50.0\t\tUnknown\t\tGreen\nB5\t50.0\t\tUnknown\t\tBlue\nB5\t50.0\t\tUnknown

## Set up the Source Labware

Here, we'll determine how much of each source material (in this case the food colourings) is required to run the protocol, and then add the required sources to the source labware layout.

For each food colouring type, we'll search the Destination labware layout and find out the total volume required for that colourant. Then, we'll divide that by the aliquot volume to find the number of aliquots needed, and add that many aliquots to the source labware layout.

In [18]:
###########################
# Set up source materials #
###########################

# For each colourant
for colourant in Food_Colouring:
    # Get the total volume of colourant in the destination layout
    colourant_total_volume = Destination_Labware_Layout.get_total_volume_of_liquid(colourant)
    
    # Calculate the number of aliquots needed
    aliquots_needed = math.ceil(colourant_total_volume / Food_Colouring_Aliquot_Volume)
    
    # Add that many aliquots to the source layout
    for aliquot_n in range(0, aliquots_needed):
        # Get the next empty well
        well = Source_Labware_Layout.get_next_empty_well()
        
        # Add content to the empty well
        Source_Labware_Layout.add_content(
            Well = well,
            Reagent = colourant,
            Volume = 500 # uL
        )

And once again the `.print` method can be used to check what has been added to the labware layout object

In [19]:
Source_Labware_Layout.print()

Information for Source Labware
Plate Type: opentrons_24_aluminumblock_nest_1.5ml_snapcap
Well	Volume(uL)	Liquid Class	Reagent
A1	500.0		Unknown		Green
A2	500.0		Unknown		Green
A3	500.0		Unknown		Blue
A4	500.0		Unknown		Blue
A5	500.0		Unknown		Red
A6	500.0		Unknown		Red
B1	500.0		Unknown		Purple
B2	500.0		Unknown		Purple


'A1\t500.0\t\tUnknown\t\tGreen\nA2\t500.0\t\tUnknown\t\tGreen\nA3\t500.0\t\tUnknown\t\tBlue\nA4\t500.0\t\tUnknown\t\tBlue\nA5\t500.0\t\tUnknown\t\tRed\nA6\t500.0\t\tUnknown\t\tRed\nB1\t500.0\t\tUnknown\t\tPurple\nB2\t500.0\t\tUnknown\t\tPurple\n'

Now let's add the food colouring to our source layout object. For this example, we'll simply pre-define where the source materials will be located.

Because we've defined which wells are available, we can use the `get_next_empty_well` method, which will simply return the next available well with no content.

In [20]:
########################
# Add source materials #
########################

for colourant in Food_Colouring:
    well = Source_Labware_Layout.get_next_empty_well()

    Source_Labware_Layout.add_content(
        Well = well,
        Reagent = colourant,
        Volume = 500 # uL
    )

## Load Labware

Now that we have the labware defined, we can load the labware to the Opentrons deck. This simply means letting the Opentrons know what labware is available, and which deck position it should be placed on. We can use `OTP.next_empty_slot` to get the next deck slot with nothing currently occupying that space.

Note that once again this piece of code uses the `protocol` object, and so won't work until we put it into the `run` function.

In [21]:
################
# Load labware #
################

Source_Labware = OTP.load_labware_from_layout(
    Protocol = protocol,
    Labware_Layout = Source_Labware_Layout,
    deck_position = OTP.next_empty_slot(protocol),
)

Destination_Labware = OTP.load_labware_from_layout(
    Protocol = protocol,
    Labware_Layout = Destination_Labware_Layout,
    deck_position = OTP.next_empty_slot(protocol),
    custom_labware_dir = Custom_Labware_Directory
)

## Determine the Transfer Steps

We'll now use the `OTP.dispense_from_aliquots` function to help us determine the transfer steps to be performed by the Opentrons, and then use those to calculate the number of pipette tips required.

We're using the `OTP.dispense_from_aliquots` function here as we need to transfer liquid from a group of aliquots (the two aliquots of each food colourant) into a list of destination wells. If you need to define a set of transfer steps from one well to another in a specific way, then [`OTP.transfer_liquids`](https://github.com/intbio-ncl/BiomationScripterLib/blob/main/docs/OTProto.md#function-transfer_liquids) might be more useful.

`OTP.dispense_from_aliquots` also has a lot more options for customising the liquid handling than we'll use here (such as aspiration speed, touch tip action, etc.). To see the full list of arguments the function takes, see [the documentation](https://github.com/intbio-ncl/BiomationScripterLib/blob/main/docs/OTProto.md#function-dispense_from_aliquots).

As we just need to calculate the transfer actions at the moment, `Calculate_Only` is set to `True`. When we actually need to perform the liquid handling, we'll use `Calculate_Only = False`.

In [22]:
#############################
    # Caclulcate transfer steps #
    #############################

    # Dictionary to keep track of how many tips are needed

    tips_required = {
        "p20": 0,
        "p300": 0,
        "p1000": 0,
    }
    
    # Dictionaries to store transfer lists, source locations, and destination locations
    transfer_lists = {}
    source_locations = {}
    destination_locations = {}
    

    # For each colourant
    for colourant in Food_Colouring:

        # Get the locations of the colourant aliquots

        ## This gets the wells as strings (e.g. "A1", "A2")
        colourant_source_wells = Source_Labware_Layout.get_wells_containing_liquid(colourant)

        ## This converts the wells to Opentrons locations, which presents the Opentrons with XYZ co-ordinates
        colourant_source_locations = OTP.get_locations(
            Labware = Source_Labware,
            Wells = colourant_source_wells
        )

        # Get destination locations which contain the colourant

        ## This gets the wells as strings (e.g. "A1", "A2")
        destination_wells = Destination_Labware_Layout.get_wells_containing_liquid(colourant)

        ## This converts the wells to Opentrons locations, which presents the Opentrons with XYZ co-ordinates
        colourant_destination_locations = OTP.get_locations(
            Labware = Destination_Labware,
            Wells = destination_wells
        )

        # Get a list of transfer volumes
        transfer_volumes = [Destination_Labware_Layout.get_volume_of_liquid_in_well(colourant, well) for well in destination_wells]

        # Store this info in the dictionaries
        transfer_lists[colourant] = transfer_volumes.copy()
        source_locations[colourant] = colourant_source_locations.copy()
        destination_locations[colourant] = colourant_destination_locations.copy()
        
        
        # Calculate the transfer list for the colourant
        transfer_volumes, aliquot_source_order, destination_locations = OTP.dispense_from_aliquots(
            Protocol = protocol,
            Transfer_Volumes = transfer_volumes,
            Aliquot_Source_Locations = colourant_source_locations,
            Destinations = colourant_destination_locations,
            new_tip = True, # This will use a new tip for every transfer
            mix_after = (10, 20), # This will mix the destination well by pipetting up and down 10 times with a volumes of 20 uL
            Calculate_Only = True
        )

        # Calculate number of tips needed
        p20, p300, p1000 = OTP.calculate_tips_needed(
            protocol = protocol,
            transfers = transfer_volumes,
            new_tip = True,
        )

        tips_required["p20"] += p20
        tips_required["p300"] += p300
        tips_required["p1000"] += p1000

IndentationError: unexpected indent (Temp/ipykernel_38332/2882051696.py, line 7)

## Load Pipette Tips

Now, we'll finally load the pipette tips using the tips required variables we set up in the previous code block

In [ ]:
#####################
# Load pipette tips #
#####################

# Load tips for the p20
OTP.add_tip_boxes_to_pipettes(
    Protocol = protocol,
    Pipette_Type = "p20",
    Tip_Type = "opentrons_96_tiprack_20ul",
    Tips_Needed = tips_required["p20"]
)

# Load tips for the p300
OTP.add_tip_boxes_to_pipettes(
    Protocol = protocol,
    Pipette_Type = "p300",
    Tip_Type = "opentrons_96_tiprack_300ul",
    Tips_Needed = tips_required["p300"]
)

## Liquid Handling

The last step is to actually perform the liquid handling. We'll use the `OTP.dispense_from_aliquots` function from earlier, but run it with `Calculate_Only = False` this time.

In [ ]:
###################
# Liquid Handling #
###################

for colourant in Food_Colouring:
    OTP.dispense_from_aliquots(
        Protocol = protocol,
        Transfer_Volumes = transfer_lists[colourant],
        Aliquot_Source_Locations = source_locations[colourant],
        Destinations = destination_locations[colourant],
        new_tip = True, # This will use a new tip for every transfer
        mix_after = (10, 20), # This will mix the destination well by pipetting up and down 10 times with a volumes of 20 uL
        Calculate_Only = False
    )

## Joining Everything Together

The code below is everything from above, put together into the `run` function:

In [ ]:
def run(protocol):
    #######################
    # General Information #
    #######################

    # Define the types of pipettes, and their location
    Pipettes = {
        "left": "p20_single_gen2",
        "right": "p300_single_gen2"
    }

    # For each pipette, define the pipette tips available to them
    Pipette_Tips = {
        "left": "opentrons_96_tiprack_20ul",
        "right": "opentrons_96_tiprack_300ul"
    }

    # List of source material

    Food_Colouring = [
        "Green",
        "Blue",
        "Red",
        "Purple"
    ]

    Food_Colouring_Aliquot_Volume = 500 # uL
    
    #################
    # Load Pipettes #
    #################

    for position in Pipettes.keys():
        protocol.load_instrument(
            Pipettes[position],
            position
        )
        
    ##################
    # Set up labware #
    ##################
    Custom_Labware_Directory = "../../../../Resources/For docs/custom_labware"

    Tube_Rack_Type = "opentrons_24_aluminumblock_nest_1.5ml_snapcap"
    Destination_Type = "greiner655087_96_wellplate_340ul"
    
    ##########################
    # Set up labware layouts #
    ##########################

    # Create the source object
    Source_Labware_Layout = BMS.Labware_Layout(
        Name = "Source Labware",
        Type = Tube_Rack_Type
    )

    # Define the labware's format (i.e. number of rows and columns)
    source_rows, source_columns = OTP.get_labware_format(
        labware_api_name = Source_Labware_Layout.type,
    )
    Source_Labware_Layout.define_format(source_rows, source_columns)
    
    # Create the destination object
    Destination_Labware_Layout = BMS.Labware_Layout(
        Name = "Destination Labware",
        Type = Destination_Type
    )

    # Define the labware's format (i.e. number of rows and columns)
    destination_rows, destination_columns = OTP.get_labware_format(
        labware_api_name = Destination_Labware_Layout.type,
        custom_labware_dir = Custom_Labware_Directory
    )
    Destination_Labware_Layout.define_format(destination_rows, destination_columns)
    
    Source_Labware_Layout.set_available_wells()
    Destination_Labware_Layout.set_available_wells()
    
    ################################
    # Define the destination state #
    ################################

    # Set up an empty list in which the colour mixtures required will be added
    Colour_Mixtures = []

    # Iterate through the different ratios
    # Iterate through the list of source colours provided to create the list of mixtures.
    for colour_1 in Food_Colouring:
        for colour_2 in Food_Colouring:
            # Ignore situations where colour_1 and colour_2 are the same
            if colour_1 == colour_2:
                continue
            else:
                # Add the two colours to the list of mixtures to prepare
                Colour_Mixtures.append([colour_1, colour_2])

    # Here, we'll print to OUT all of the mixtures which will be prepared
    for c in Colour_Mixtures:
        print(c)
        
    # Define the final destination state

    # For 3 replicates
    for rep in range(0, 3):
        # For each mixture
        for colour_1, colour_2 in Colour_Mixtures:

            # Get the next empty well
            well = Destination_Labware_Layout.get_next_empty_well()

            # Add the first colourant
            Destination_Labware_Layout.add_content(
                Well = well,
                Reagent = colour_1,
                Volume = 50 # uL
            )

            # And then the second
            Destination_Labware_Layout.add_content(
                Well = well,
                Reagent = colour_2,
                Volume = 50 # uL
            )

            # Can also label the well
            Destination_Labware_Layout.add_well_label(
                Well = well,
                Label = "Mixture: {}-{} Rep {}".format(colour_1, colour_2, rep)
            )
            
    Destination_Labware_Layout.print()
    
    ###########################
    # Set up source materials #
    ###########################

    # For each colourant
    for colourant in Food_Colouring:
        # Get the total volume of colourant in the destination layout
        colourant_total_volume = Destination_Labware_Layout.get_total_volume_of_liquid(colourant)

        # Calculate the number of aliquots needed
        aliquots_needed = math.ceil(colourant_total_volume / Food_Colouring_Aliquot_Volume)

        # Add that many aliquots to the source layout
        for aliquot_n in range(0, aliquots_needed):
            # Get the next empty well
            well = Source_Labware_Layout.get_next_empty_well()

            # Add content to the empty well
            Source_Labware_Layout.add_content(
                Well = well,
                Reagent = colourant,
                Volume = 500 # uL
            )
            
    Source_Labware_Layout.print()
    
    ########################
    # Add source materials #
    ########################

    for colourant in Food_Colouring:
        well = Source_Labware_Layout.get_next_empty_well()

        Source_Labware_Layout.add_content(
            Well = well,
            Reagent = colourant,
            Volume = 500 # uL
        )
        
    ################
    # Load labware #
    ################

    Source_Labware = OTP.load_labware_from_layout(
        Protocol = protocol,
        Labware_Layout = Source_Labware_Layout,
        deck_position = OTP.next_empty_slot(protocol),
    )

    Destination_Labware = OTP.load_labware_from_layout(
        Protocol = protocol,
        Labware_Layout = Destination_Labware_Layout,
        deck_position = OTP.next_empty_slot(protocol),
        custom_labware_dir = Custom_Labware_Directory
    )
    
    #############################
    # Caclulcate transfer steps #
    #############################

    # Dictionary to keep track of how many tips are needed

    tips_required = {
        "p20": 0,
        "p300": 0,
        "p1000": 0,
    }
    
    # Dictionaries to store transfer lists, source locations, and destination locations
    transfer_lists = {}
    source_locations = {}
    destination_locations = {}
    

    # For each colourant
    for colourant in Food_Colouring:

        # Get the locations of the colourant aliquots

        ## This gets the wells as strings (e.g. "A1", "A2")
        colourant_source_wells = Source_Labware_Layout.get_wells_containing_liquid(colourant)

        ## This converts the wells to Opentrons locations, which presents the Opentrons with XYZ co-ordinates
        colourant_source_locations = OTP.get_locations(
            Labware = Source_Labware,
            Wells = colourant_source_wells
        )

        # Get destination locations which contain the colourant

        ## This gets the wells as strings (e.g. "A1", "A2")
        destination_wells = Destination_Labware_Layout.get_wells_containing_liquid(colourant)

        ## This converts the wells to Opentrons locations, which presents the Opentrons with XYZ co-ordinates
        colourant_destination_locations = OTP.get_locations(
            Labware = Destination_Labware,
            Wells = destination_wells
        )

        # Get a list of transfer volumes
        transfer_volumes = [Destination_Labware_Layout.get_volume_of_liquid_in_well(colourant, well) for well in destination_wells]

        # Store this info in the dictionaries
        transfer_lists[colourant] = transfer_volumes.copy()
        source_locations[colourant] = colourant_source_locations.copy()
        destination_locations[colourant] = colourant_destination_locations.copy()
        
        
        # Calculate the transfer list for the colourant
        transfer_volumes, aliquot_source_order, colourant_destination_locations = OTP.dispense_from_aliquots(
            Protocol = protocol,
            Transfer_Volumes = transfer_volumes,
            Aliquot_Source_Locations = colourant_source_locations,
            Destinations = colourant_destination_locations,
            new_tip = True, # This will use a new tip for every transfer
            mix_after = (10, 20), # This will mix the destination well by pipetting up and down 10 times with a volumes of 20 uL
            Calculate_Only = True
        )

        # Calculate number of tips needed
        p20, p300, p1000 = OTP.calculate_tips_needed(
            protocol = protocol,
            transfers = transfer_volumes,
            new_tip = True,
        )

        tips_required["p20"] += p20
        tips_required["p300"] += p300
        tips_required["p1000"] += p1000
        
        

    #####################
    # Load pipette tips #
    #####################

    # Load tips for the p20
    OTP.add_tip_boxes_to_pipettes(
        Protocol = protocol,
        Pipette_Type = "p20",
        Tip_Type = "opentrons_96_tiprack_20ul",
        Tips_Needed = tips_required["p20"]
    )

    # Load tips for the p300
    OTP.add_tip_boxes_to_pipettes(
        Protocol = protocol,
        Pipette_Type = "p300",
        Tip_Type = "opentrons_96_tiprack_300ul",
        Tips_Needed = tips_required["p300"]
    )

    ###################
    # Liquid Handling #
    ###################

    for colourant in Food_Colouring:
        OTP.dispense_from_aliquots(
            Protocol = protocol,
            Transfer_Volumes = transfer_lists[colourant],
            Aliquot_Source_Locations = source_locations[colourant],
            Destinations = destination_locations[colourant],
            new_tip = True, # This will use a new tip for every transfer
            mix_after = (10, 20), # This will mix the destination well by pipetting up and down 10 times with a volumes of 20 uL
            Calculate_Only = False
        )

## Simulation

The code below can be used to simulate the protocol. Note that this **MUST** be removed before trying to run the script on the Opentrons.

In [ ]:
#####################################################################
# Use this cell if simulating the protocol, otherwise comment it out #
#####################################################################

#########################################################################################################
# IMPORTANT - the protocol will not upload to the opentrons if this cell is not commented out or removed #
#########################################################################################################

from opentrons import simulate as OT2 # This line simulates the protocol
# Get the correct api version
protocol = OT2.get_protocol_api(metadata["apiLevel"])
# Home the pipetting head
protocol.home()
# Call the 'run' function to run the protocol
p = run(protocol)
for line in protocol.commands():
    print(line)